In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tkinter
import simpy
import random
import time 
import pandas as pd
import csv
from sklearn import cluster

In [2]:
class Location:
    def __init__(self, name=None, distance=None, river=None, distance_cluster=None):
        self.name = name
        self.distance = distance
        self.river = river
        self.cluster = cluster

In [3]:
class Call:
    id_call = 0 
    def __init__(self, ID=None, initial_time=None, final_time=None, status=None, location=None):
        Call.id_call = Call.id_call + 1
        self.ID = Call.id_call
        self.initial_time = initial_time
        self.final_time = final_time
        self.status = status 
        self.location = location
         
        
    def detail(self):
        print('Call from {} received at {}'.format(self.location.name, self.initial_time))
        
    def generate_Call(self, env, beta, distance_SB, p, AB, SB_negro, SB_amazonas, boat_list, boat_list_amazonas, boat_list_negro, lastCall=None):
        self.location = Location()
        self.location.name = random.choices([*distance], p)[0]
        self.location.distance = distance[self.location.name][0]
        self.location.river = distance[self.location.name][1]
        self.location.cluster = distance_SB[self.location.name][2]
        

        if (lastCall==None):
            self.initial_time = float (np.random.exponential(scale=beta, size=1))
            yield env.timeout(self.initial_time)
        else:    
            self.initial_time = float (lastCall.initial_time + np.random.exponential(scale=beta, size=1))
            yield env.timeout(self.initial_time)

        print('%7.4f Call %d received from %s at a distance of %d on the river %s at a distance of the small base %d on the cluster %d' % (env.now, self.ID, self.location.name, self.location.distance, self.location.river, distance_SB[self.location.name][0], self.location.cluster))
        print("temps estimé dans la version initiale : {}".format(2*(self.location.distance/boat_list[0].speed) + 0.25))
        if(self.location.river == "amazonas"): 
            yield env.process(trajet(env, boat_list_amazonas, AB, SB_amazonas, "cluster_amazonas", self, distance_SB))       
        else: 
            yield env.process(trajet(env, boat_list_negro, AB, SB_negro, "cluster_negro", self, distance_SB))
        
        self.status = 1
        

In [4]:
def trajet(env, boat_list_river, AB, resource, cluster_name, self, distance_SB): 
    with resource.request() as req : 
        yield req
        time = 0 
        time_to_location = 0
        wait = env.now - self.initial_time #waiting time 
        start = env.now
        i = 0 
        if(boat_list_river[self.location.cluster].status == 0): 
            i = self.location.cluster
        else: 
            k = self.location.cluster-1 
            l = self.location.cluster+1
            if(k<0): 
                k = 0
            if(l == len(boat_list_river)): 
                l = len(boat_list_river)-1
            recherche = True 
            while(recherche):
                #print("k: {} l: {}".format(k,l))
                if(k>=0 or l<len(boat_list_river)):
                    plus_proche_base = min(distance_SB[cluster_name+str(boat_list_river[k].cluster)], 
                                           distance_SB[cluster_name+str(boat_list_river[l].cluster)])
                    if(plus_proche_base == distance_SB[cluster_name+str(boat_list_river[k].cluster)]): 
                        if(boat_list_river[k].status == 0):
                            recherche = False 
                            i = k
                        else: 
                            k = k-1
                    else: 
                        if(boat_list_river[l].status == 0):
                            recherche = False 
                            i = l
                        else: 
                            if(l != len(boat_list_river) - 1):
                                l =l+1
                            else: 
                                l = 0
        #print('%7.4f Departure of the simple boat %s %d from cluster %d'%(env.now, self.location.river, boat_list_river[i].ID, boat_list_river[i].cluster))
        if(i != len(boat_list_river) and boat_list_river[i].status == 0):
            boat_list_river[i].status = 1
            if(boat_list_river[i].cluster == self.location.cluster): 
                time_to_location = distance_SB[self.location.name][0]/boat_list_river[i].speed
                time = time_to_location + self.location.distance/boat_list_river[i].speed
            else:
                time_to_location = abs(distance_SB[cluster_name+str(boat_list_river[i].cluster)] - 
                                       distance_SB[cluster_name+str(self.location.cluster)])/boat_list_river[i].speed + distance_SB[self.location.name][0]/boat_list_river[i].speed
                time = time_to_location + self.location.distance/boat_list_river[i].speed
            initial_AB = env.now
            with AB.request() as req_AB:
                yield req_AB
                wait_AB = env.now - initial_AB 
                #print('%7.4f Departure of the advanced boat'%(env.now))
                j = 0 
                while(j != len(boat_list) and boat_list[j].status != 0): #while you don't have a boat available 
                    j = j + 1
                if(j != len(boat_list) and boat_list[j].status == 0):
                    boat_list[j].status = 1
                    intervention = Intervention()
                    intervention.boat_SB = boat_list_river[i]
                    intervention.boat_AB = boat_list[j]
                    intervention.call = self
                    intervention.waited_time = wait 
                    intervention.TOS = 0.25
                    if(self.location.distance/boat_list[j].speed < distance_SB[self.location.name][0]/boat_list_river[i].speed):
                        t = 2*(self.location.distance/boat_list[j].speed)
                        boat_list_river[i].status = 0 
                        distance_to_base = distance_SB[cluster_name+str(boat_list_river[i].cluster)]
                    else:
                        if(env.now > start + time ): 
                            print('appel traité par le boat simple')
                            distance_to_base = (env.now - start - time )*boat_list_river[i].speed
                            t = 0
                        else: 
                            reste = start + time - env.now #temps qu'il reste pour que le simple boat finisse le trajet 
                            if(reste<time-time_to_location): #si je suis déjà arrivé à la location et que je suis reparti  
                                d = reste*boat_list_river[i].speed #c'est la distance qu'il reste à parcourir au simple boat, autrement dit la distance entre le simple boat et la base principale 
                                t = 2*(d/(boat_list_river[i].speed + boat_list[j].speed))
                            else: #si je suis entre le cluster et la location 
                                d1 = (time_to_location-(time-reste))*boat_list[j].speed #(time - reste) represente la distance deja parcouru par le simple boat. Et comme reste > time-TTL, TTL-(time-reste) est positif
                                t = 2*(time_to_location-(time-reste)) + 2*((self.location.distance-d1)/(boat_list_river[i].speed + boat_list[j].speed)) #on fait distance-d1 car c'est la distance que l'AB a deja parcouru 
                            distance_to_base = boat_list[j].speed*(t/2)
                    print("temps réel d'éxécution {}".format(t + intervention.TOS + wait_AB)) 
                    intervention.travel_time = t/2
                    intervention.response_time = intervention.travel_time + intervention.TOS
                    intervention.road_time = t + intervention.TOS + wait_AB
                    inter_list.append(intervention)
                    self.final_time = self.initial_time + intervention.road_time 
                    yield env.timeout(intervention.travel_time + intervention.TOS)
                    #print('%7.4f Call %d: echange en cours' % (env.now, self.ID))
                    echange = env.now
                    yield env.timeout(intervention.travel_time)
                    print('%7.4f Call %d: Finished' % (env.now, self.ID))
                    boat_list[j].status = 0
        distance_to_cluster = abs(distance_SB[cluster_name+str(boat_list_river[i].cluster)] - distance_to_base)
        #print("distance_to_cluster : {}".format(distance_to_cluster))
        #print("temps prevu pour retour : {}".format(distance_to_cluster/boat_list_river[i].speed))
        if(env.now > echange + distance_to_cluster/boat_list_river[i].speed): 
            #print('%7.4f boat %s %d : déjà arrivée a la base'%(env.now, self.location.river, boat_list_river[i].ID))
            boat_list_river[i].status = 0 
        else:
            #print('%7.4f boat %s %d : en cours de retour a la base '%(env.now, self.location.river, boat_list_river[i].ID))
            distance_restante = abs(distance_to_cluster - (env.now - echange)*boat_list_river[i].speed)
            yield env.timeout(distance_restante/boat_list_river[i].speed)
            #print('%7.4f boat %s %d : arrivée a la base'%(env.now, self.location.river, boat_list_river[i].ID))
            boat_list_river[i].status = 0 

In [5]:
class Intervention: 
    nb_inter = 0 
    def __init__(self, travel_time=None, TOS=None, response_time=None, road_time = None, waited_time=None, boat_SB=None, boat_AB=None, call=None):
        Intervention.nb_inter = Intervention.nb_inter + 1 
        self.ID = Intervention.nb_inter
        self.travel_time = travel_time
        self.TOS = TOS
        self.response_time = response_time 
        self.road_time = road_time 
        self.waited_time = waited_time 
        self.boat_SB = boat_SB
        self.boat_AB = boat_AB
        self.call = call
        
    def to_dict(self): 
        return {
            'Cluster du call': self.call.location.cluster,
            'Simple Boat': self.boat_SB.ID,
            'Cluster boat': self.boat_SB.cluster,
            'Advanced Boat': self.boat_AB.ID, 
            'Call traited': self.call.ID,
            'Call location': self.call.location.name, 
            'travel time': self.travel_time,
            'response time': self.response_time,
            'waited time': self.waited_time,
            'total time': self.road_time, 
        }

In [6]:
class Boat: 
    nb_boat = 0 
    def __init__(self, env, speed=None, status=None, cluster=None): 
        Boat.nb_boat = Boat.nb_boat + 1 
        self.ID = Boat.nb_boat 
        self.speed = speed 
        self.status = status 
        self.cluster = cluster

In [7]:
def clustering(distance, nb_boat_simple): 
    distance_negro = []
    distance_amazonas = []
    for i in range(len(distance)): #on regarde le fleuve de chaque location pour permettre de faire un cluster par bras de fleuve  
        if(distance[[*distance][i]][1] == 'amazonas'):
            distance_amazonas.append([0,distance[[*distance][i]][0]]) #on est obligé de mettre [0, distance] pour la fonction de clustering 
        else: 
            distance_negro.append([0,distance[[*distance][i]][0]])
    #on définit les paramètres du calcul des clusters 
    kmeans_negro = cluster.KMeans(n_clusters=nb_boat_simple, max_iter=1000) 
    kmeans_amazonas = cluster.KMeans(n_clusters=nb_boat_simple, max_iter=1000)
    
    #on réalise les calculs 
    kmeans_amazonas.fit(distance_amazonas)
    kmeans_negro.fit(distance_negro) 
    
    #on calcule pour chaque fleuve la distance des locations par rapport au nouveau centre 
    for i in range(len(kmeans_amazonas.labels_)): 
        distance_amazonas[i][1] = abs(distance_amazonas[i][1]-kmeans_amazonas.cluster_centers_[kmeans_amazonas.labels_[i]][1])
    for i in range(len(kmeans_negro.labels_)): 
        distance_negro[i][1] = abs(distance_negro[i][1]-kmeans_negro.cluster_centers_[kmeans_negro.labels_[i]][1])
        
    #on recrée le dictionnaire initial avec les nouvelles distances. Comme c'est dans le même ordre qu'à l'aller, le
    #dictionnaire se recrée tout seul automatiquement 
    distance_SB = {}
    for loc in [*distance]: 
        if(distance[loc][1] == 'amazonas'): 
            distance_SB[loc] = (distance_amazonas[0][1], 'amazonas', kmeans_amazonas.labels_[0])
            kmeans_amazonas.labels_ = np.delete(kmeans_amazonas.labels_, 0)
            del distance_amazonas[0] #on est obligé de faire pop l'élément pour pouvoir recréer le dictionnaire dans le même ordre
        else: 
            distance_SB[loc] = (distance_negro[0][1], 'negro', kmeans_negro.labels_[0])
            kmeans_negro.labels_ = np.delete(kmeans_negro.labels_, 0)
            del distance_negro[0]
    for i in range(len(kmeans_amazonas.cluster_centers_)): 
        distance_SB["cluster_amazonas"+str(i)] = kmeans_amazonas.cluster_centers_[i][1]
    for i in range(len(kmeans_negro.cluster_centers_)): 
        distance_SB["cluster_negro"+str(i)] = kmeans_negro.cluster_centers_[i][1]
    return(distance_SB)

In [8]:
def source(env, AB, SB_negro, SB_amazonas, distance_SB, p, boat_list, boat_list_amazonas, boat_list_negro, call_list): 
    yield env.timeout(0)
    print('%7.4f Start' % (env.now))
    for i in range(nb_call):
        call_list.append(Call())
        if i == 0: 
            env.process(call_list[i].generate_Call(env, coeff_call, distance_SB, p, AB, SB_negro, SB_amazonas, boat_list, boat_list_amazonas, boat_list_negro))
        else: 
            env.process(call_list[i].generate_Call(env, coeff_call, distance_SB, p, AB, SB_negro, SB_amazonas, boat_list, boat_list_amazonas, boat_list_negro, call_list[i-1]))

In [9]:
#Define the variables
p_2015 = [0.07246376812, 0.03623188406, 0.02898550725, 0.05797101449, 0.007246376812, 0.1086956522, 0.007246376812, 0.1449275362, 0.1231884058, 0.04347826087, 0.04347826087, 0.09420289855, 0.04347826087, 0.1231884058, 0.0652173913]
p_2016 = [0.04255319149, 0.01418439716, 0.007092198582, 0.07092198582, 0.04964539007, 0.09929078014, 0.01418439716, 0.06382978723, 0.09929078014, 0.09219858156, 0.134751773, 0.07092198582, 0.03546099291, 0.1843971631, 0.02127659574]
distance = {'N SRA CARMO':(100, 'amazonas'), 'N SRA P. SOCORRO':(90, 'amazonas'), 'BONSUCESSO':(80, 'amazonas'), 
         'ASS. NAZARE':(70, 'amazonas'), 'GUAJARA':(60, 'negro'), 'ARUAU':(100, 'negro'), 'MIPINDIAU':(90, 'negro'), 'CUIEIRAS':(75, 'negro'), 
         'SANTA MARIA':(60, 'negro'), 'JARAQUI':(52, 'negro'), 'COSTA DOS ARARAS':(50, 'amazonas'), 
         'COSTA DO TATU':(30, 'negro'), 'AGROVILA':(20, 'negro'), 'N. SRA. FATIMA':(15, 'negro'), 'LIVRAMENTO':(15, 'negro')}
nb_boat_avanced = 3
nb_boat_simple = 1
nb_call = 164
coeff_call = 38
nb_repet = 100
inter_list = []
df = []
call_list = []
interval = []
test = []

In [10]:
def model(): 
    env = simpy.Environment()
    boat_list_amazonas = []
    boat_list_negro = []
    boat_list = []
    distance_SB = clustering(distance, nb_boat_simple)
    for i in range(nb_boat_avanced): 
        boat_list.append(Boat(env,46,0))
    for i in range(nb_boat_simple): 
        boat_list_amazonas.append(Boat(env,30,0,i))
        boat_list_negro.append(Boat(env,30,0,i))
    AB = simpy.Resource(env, capacity=nb_boat_avanced)
    SB_negro = simpy.Resource(env, capacity=nb_boat_simple)
    SB_amazonas = simpy.Resource(env, capacity=nb_boat_simple)
    env.process(source(env, AB, SB_negro, SB_amazonas, distance_SB, p_2015, boat_list, boat_list_amazonas, boat_list_negro, call_list))
    env.run()

#Allow to launch nb_repet of repetitions 
for i in range(nb_repet): 
    model()
    df.append(pd.DataFrame.from_records([inter.to_dict() for inter in inter_list]))
    
    #allow to save the gaps between each call 
    gaps = [] 
    gaps.append(call_list[0].initial_time)
    for i in range (1,len(call_list)-1):
        gaps.append(call_list[i].initial_time-call_list[i-1].initial_time)
    mean_gaps = sum(gaps)/len(gaps)
    interval.append(mean_gaps)
    
    for inter in inter_list: 
        test.append(inter.road_time)
    
    #reset the global variables 
    inter_list = []
    call_list = []

 0.0000 Start
36.9716 Call 1 received from SANTA MARIA at a distance of 60 on the river negro at a distance of the small base 8 on the cluster 0
temps estimé dans la version initiale : 2.858695652173913


NameError: name 'boat_list' is not defined

In [ ]:
#data statistic 

waited_mean = []
travel_time_mean = []
response_time_mean = []
total_time_mean = []
sum_location = 0 
for i in range(nb_repet): 
    #print("temps moyen d'attente simu {} : {}".format(i,df[i]["waited time"].mean()))
    waited_mean.append(df[i]["waited time"].mean())
    #print("temps moyen de trajet simu {} : {}".format(i,df[i]["travel time"].mean()))
    travel_time_mean.append(df[i]["travel time"].mean())
    #print("temps moyen de réponse simu {} : {}".format(i,df[i]["response time"].mean()))
    response_time_mean.append(df[i]["response time"].mean())
    #print("temps moyen total simu {} : {}".format(i,df[i]["total time"].mean()))
    total_time_mean.append(df[i]["total time"].mean())
    #print(df[i]["Call location"].value_counts() / nb_call)
    sum_location = sum_location + df[i]["Call location"].value_counts() 
    
    
print("temps moyen d'attente pour {} simulation : {}".format(nb_repet, sum(waited_mean)/len(waited_mean)))
print("temps moyen de trajet pour {} simulation : {}".format(nb_repet, sum(travel_time_mean)/len(travel_time_mean)))
print("temps moyen de reponse pour {} simulation : {}".format(nb_repet, sum(response_time_mean)/len(response_time_mean)))
print("temps moyen total pour {} simulation : {}".format(nb_repet, sum(total_time_mean)/len(total_time_mean)))
print("probabilités pour chaque location ")
print(sum_location / (nb_repet*nb_call))


In [ ]:
#Real data 
df = pd.read_csv("sample-2015-wo.csv", sep=",", encoding="latin-1")
mean_inter_HS = np.mean(df["INTERVAL_HS"])
mean_duration_HS = np.mean(df["DURACAO_HS"])

In [ ]:
#Gaps analysis 
mean_interval = sum(interval)/len(interval)
print("Mean : {}  Max : {}  Min : {}".format(mean_interval, max(interval), min(interval)))
print("Real data mean : {}".format(mean_inter_HS))

plt.title("Gaps between call average")
plt.plot(interval)
plt.axhline(y=max(interval), color='r', linestyle='--')
plt.axhline(y=min(interval), color='r', linestyle='--')
plt.axhline(y=mean_inter_HS, color='y', linestyle='-')
plt.axhline(y=mean_interval, color='g', linestyle='-')
plt.show()

In [ ]:
#Time graphs 
print("Mean : {}  Max : {}  Min : {}".format(sum(total_time_mean)/len(total_time_mean), max(total_time_mean), min(total_time_mean)))
print("Real data mean : {}".format(mean_duration_HS))
plt.plot(total_time_mean)
plt.title("Total time average ")
plt.axhline(y=max(total_time_mean), color='r', linestyle='--')
plt.axhline(y=min(total_time_mean), color='r', linestyle='--')
plt.axhline(y=mean_duration_HS, color='y', linestyle='-')
plt.axhline(y=sum(total_time_mean)/len(total_time_mean), color='g', linestyle='-')
plt.show()

In [ ]:
index = []
simul_data = []
for loc in [*distance]: 
    index.append(loc)
    simul_data.append(sum_location[loc]/(nb_repet*nb_call))

result = pd.DataFrame({'real': p_2015, 'simu': simul_data}, index=index)
result.plot.bar(rot=80)

In [ ]:
tab = [0,1]
len(tab)